In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import urllib
import requests
import re
import nltk
import datetime
import re

import pandas as pd

In [2]:
ratings = []

First, we'll try to get the IMDB episode ratings for just season one:

In [2]:
result = requests.get('http://www.imdb.com/title/tt0106145/episodes?season=1')
result

In [33]:
soup = BeautifulSoup(result.content, 'lxml')

In [34]:
soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///title/tt0106145?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var ue_t0=window.ue_t0||+new Date();</script>
<script type="text/javascript">
                var ue_mid = "A1EVAM02EL8SFB"; 
                var ue_sn = "www.imdb.com";  
                var ue_furl = "fls-na.amazon.com";
                var ue_sid = "000-0000000-0000000";
                var ue_id = "0A6PCWZ9S521WXQ2NV8F";
                (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:arguments};c.ueLogError(k);return false}b.skipTr

In [35]:
soup.find('div', {'class': 'info'})

<div class="info" itemprop="episodes" itemscope="" itemtype="http://schema.org/TVEpisode">
<meta content="1" itemprop="episodeNumber"/>
<div class="airdate">
            3 Jan. 1993
    </div>
<strong><a href="/title/tt0108214/?ref_=ttep_ep1" itemprop="name" title="Emissary">Emissary</a></strong>
<div class="ipl-rating-widget">
<div class="ipl-rating-star ">
<span class="ipl-rating-star__star">
<svg class="ipl-star-icon" fill="#000000" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
<path d="M0 0h24v24H0z" fill="none"></path>
<path d="M12 17.27L18.18 21l-1.64-7.03L22 9.24l-7.19-.61L12 2 9.19 8.63 2 9.24l5.46 4.73L5.82 21z"></path>
<path d="M0 0h24v24H0z" fill="none"></path>
</svg>
</span>
<span class="ipl-rating-star__rating">7.5</span>
<span class="ipl-rating-star__total-votes">(2,552)</span>
</div>
<div class="ipl-rating-interactive ipl-rating-interactive--no-rating">
<input class="ipl-rating-interactive__state" data-reftag="ttep_ep1" data-tconst="tt010

In [160]:
# First, let's find the season:
# This will go outside the episode loop but inside the season loops

season_tag = soup.find('h3',{'id': 'episode_top'})
season_tag_text = season_tag.text.replace(u'\xa0', u' ')

pattern = re.compile(r'[0-9]')

season = re.search(pattern, season_tag_text)
season = season.group(0)

I'll need these values for each episode:

In [161]:
# Find episode title
soup.find('div', {'class': 'info'}).find('a', {'itemprop': 'name'}).text

'Emissary'

In [165]:
# Find episode number
soup.find('div', {'class': 'info'}).find('meta').attrs['content']

'1'

In [168]:
# Find airdate
soup.find('div', {'class': 'info'}).find('div', {'class': 'airdate'}).contents[0].strip()

'3 Jan. 1993'

In [169]:
# Find rating
soup.find('div', {'class': 'info'}).find('span', {'class': 'ipl-rating-star__rating'}).text

'7.5'

In [134]:
#imdb_ratings = pd.DataFrame(columns=['ep_name'])

ep_name_list = []
number_list = []
date_list = []
rating_list = []

for i, div in enumerate(soup.find_all('div', {'class': 'info'})):
    
    # append ep_name
    ep_name = div.find('a', {'itemprop': 'name'}).text
    ep_name_list.append(ep_name)
    
    # append ep number
    number = div.find('meta')
    number = number.attrs['content']
    number = int(number)
    number_list.append(number)
    
    # append airdate
    date = div.find('div', {'class': 'airdate'}).contents[0].strip()
    
    date_list.append(date)
    
    # append rating
    rating = div.find('span', {'class': 'ipl-rating-star__rating'}).text
    rating = float(rating)
    rating_list.append(rating)

Alright. Let's demo this on all soup

In [175]:
soup.find('div', {'class': 'list detail eplist'})

<div class="list detail eplist">
<div class="list_item odd">
<div class="image">
<a href="/title/tt0108214/?ref_=ttep_ep1" itemprop="url" title="Emissary"> <div class="hover-over-image zero-z-index " data-const="tt0108214">
<img alt="Emissary" class="zero-z-index" height="112" src="https://images-na.ssl-images-amazon.com/images/M/MV5BMzI0MzM3NzY5NF5BMl5BanBnXkFtZTgwNjQ1MTg0MjE@._V1_UX200_CR0,0,200,112_AL_.jpg" width="200"/>
<div>S1, Ep1</div>
</div>
</a> </div>
<div class="info" itemprop="episodes" itemscope="" itemtype="http://schema.org/TVEpisode">
<meta content="1" itemprop="episodeNumber"/>
<div class="airdate">
            3 Jan. 1993
    </div>
<strong><a href="/title/tt0108214/?ref_=ttep_ep1" itemprop="name" title="Emissary">Emissary</a></strong>
<div class="ipl-rating-widget">
<div class="ipl-rating-star ">
<span class="ipl-rating-star__star">
<svg class="ipl-star-icon" fill="#000000" height="24" viewbox="0 0 24 24" width="24" xmlns="http://www.w3.org/2000/svg">
<path d="M0 0h2

In [178]:
big_soup = soup.find('div', {'class': 'list detail eplist'})

In [186]:
ep_name_list = []
number_list = []
date_list = []
rating_list = []

for ep in big_soup.find_all('div', {'class': 'info'}):
    # append ep_name
    ep_name = ep.find('a', {'itemprop': 'name'}).text
    ep_name_list.append(ep_name)
    
    # append ep number
    number = ep.find('meta')
    number = number.attrs['content']
    number = int(number)
    number_list.append(number)
    
    # append airdate
    date = ep.find('div', {'class': 'airdate'}).contents[0].strip()
    date_list.append(date)
    
    # append rating
    rating = ep.find('span', {'class': 'ipl-rating-star__rating'}).text
    rating = float(rating)
    rating_list.append(rating)
    
pd.DataFrame(
    {'ep_name': ep_name_list,
    'rating': rating_list,
     'airdate': date_list,
     'number': number_list
    }).head()

,airdate,ep_name,number,rating
0,3 Jan. 1993,Emissary,1,7.5
1,10 Jan. 1993,Past Prologue,2,7.0
2,17 Jan. 1993,A Man Alone,3,6.9
3,24 Jan. 1993,Babel,4,6.9
4,31 Jan. 1993,Captive Pursuit,5,7.7


In [4]:
scr = []

In [5]:
for i in range(1, 8):
    request = requests.get('http://www.imdb.com/title/tt0106145/episodes?season={}'.format(i))
    scr.append(request)

In [190]:
# All seasons...
for i in range(1,8):
    print('http://www.imdb.com/title/tt0106145/episodes?season={}'.format(i))
    

http://www.imdb.com/title/tt0106145/episodes?season=1
http://www.imdb.com/title/tt0106145/episodes?season=2
http://www.imdb.com/title/tt0106145/episodes?season=3
http://www.imdb.com/title/tt0106145/episodes?season=4
http://www.imdb.com/title/tt0106145/episodes?season=5
http://www.imdb.com/title/tt0106145/episodes?season=6
http://www.imdb.com/title/tt0106145/episodes?season=7


In [6]:
scr

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [7]:
many_soups = []
for ep in scr:
    many_soups.append(BeautifulSoup(ep.content, "lxml"))

In [8]:
ep_name_list = []
number_list = []
date_list = []
rating_list = []
season_number_list = []

for season in many_soups:
    # Gets season number
    season_tag = season.find('h3',{'id': 'episode_top'})
    season_tag_text = season_tag.text.replace(u'\xa0', u' ')
    pattern = re.compile(r'[0-9]')
    season_number = re.search(pattern, season_tag_text)
    season_number = season_number.group(0) 
    
    for ep in season.find_all('div', {'class': 'info'}):
        # append ep_name
        ep_name = ep.find('a', {'itemprop': 'name'}).text
        ep_name_list.append(ep_name)
        
        # append ep number
        number = ep.find('meta')
        number = number.attrs['content']
        number = int(number)
        number_list.append(number)

        # append airdate
        date = ep.find('div', {'class': 'airdate'}).contents[0].strip()
        date_list.append(date)

        # append rating
        rating = ep.find('span', {'class': 'ipl-rating-star__rating'}).text
        rating = float(rating)
        rating_list.append(rating)
        
        season_number_list.append(season_number)

In [9]:
imdb_data = pd.DataFrame(
    {'season': season_number_list,
     'number': number_list,
    'ep_name': ep_name_list,
    'rating': rating_list,
     'airdate': date_list,
    })

In [10]:
imdb_data.head()

,airdate,ep_name,number,rating,season
0,3 Jan. 1993,Emissary,1,7.5,1
1,10 Jan. 1993,Past Prologue,2,7.0,1
2,17 Jan. 1993,A Man Alone,3,6.9,1
3,24 Jan. 1993,Babel,4,6.9,1
4,31 Jan. 1993,Captive Pursuit,5,7.7,1


In [11]:
imdb_data.describe()

,number,rating
count,173.000000,173.000000
mean,12.971098,7.509249
std,7.320271,0.782977
min,1.000000,5.400000
25%,7.000000,7.000000
50%,13.000000,7.600000
75%,19.000000,8.000000
max,26.000000,9.300000


In [12]:
imdb_data['index'] = list(range(1, 174))

In [13]:
imdb_data.head()

,airdate,ep_name,number,rating,season,index
0,3 Jan. 1993,Emissary,1,7.5,1,1
1,10 Jan. 1993,Past Prologue,2,7.0,1,2
2,17 Jan. 1993,A Man Alone,3,6.9,1,3
3,24 Jan. 1993,Babel,4,6.9,1,4
4,31 Jan. 1993,Captive Pursuit,5,7.7,1,5


In [14]:
imdb_data.to_csv('imdb_ds9_data.csv', index=False)

In [15]:
pd.read_csv('./imdb_ds9_data.csv')

,airdate,ep_name,number,rating,season,index
0,3 Jan. 1993,Emissary,1,7.5,1,1
1,10 Jan. 1993,Past Prologue,2,7.0,1,2
2,17 Jan. 1993,A Man Alone,3,6.9,1,3
3,24 Jan. 1993,Babel,4,6.9,1,4
4,31 Jan. 1993,Captive Pursuit,5,7.7,1,5
5,7 Feb. 1993,Q-Less,6,6.8,1,6
6,14 Feb. 1993,Dax,7,7.1,1,7
7,22 Feb. 1993,The Passenger,8,6.4,1,8
8,14 Mar. 1993,Move Along Home,9,5.9,1,9
9,21 Mar. 1993,The Nagus,10,7.1,1,10
